# Análise exploratória de dados

Dependendo do objetivo podemos ter mais ou menos etapas. Vamos escolher um desafio mais dificil para cobrir a maioria das possibilidades de modelagem.

No nosso exemplo vamos modelar uma regressão para explicar os preços das casas nos EUA e usa-lo para escolher as casas desvalorizadas.

    -> Identificar como a base foi construida
    -> Quais foram as regras, essas regras influeciam os dados?
    -> Devemos nos preocupar os com outliners?
    -> Ao analisar como as variáveis estão distribuidas, temos funções conhecidas?
    -> Como as funções se correlacionam, os comportamentos são os previstos?
    -> Para estudar a correlação parcial com regressões precisamos mudar a forma dos dados?
    -> Transformações logaritmicas
    -> Variáveis dummies, quando usa-las
    -> Mudando a forma funcional com polinomios
    -> Iterando váriaveis
    -> Gerando predições

Vamos Modelando uma regressão multipla para entender como diferentes variáveis afetam os preços de casas nos EUA.

Features:
- **price** - The last price the house was sold for
- **num_bed** - The number of bedrooms
- **num_bath** - The number of bathrooms (fractions mean the house has a toilet-only or shower/bathtub-only bathroom)
- **size_house** (includes basement) - The size of the house
- **size_lot** - The size of the lot
- **num_floors** - The number of floors
- **is_waterfront** - Whether or not the house is a waterfront house (0 means it is not a waterfront house whereas 1 means that it is a waterfront house)
- **condition** - How worn out the house is. Ranges from 1 (needs repairs all over the place) to 5 (the house is very well maintained)
- **size_basement** - The size of the basement
- **year_built** - The year the house was built
- **renovation_date** - The year the house was renovated for the last time. 0 means the house has never been renovated
- **zip** - The zip code
- **latitude** - Latitude
- **longitude** - Longitude
- **avg_size_neighbor_houses** - The average house size of the neighbors
- **avg_size_neighbor_lot** - The average lot size of the neighbors

# Importando os principais pacotes

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sqlite3

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
%config InlineBackend.figure_formats='svg'

# Criando a conexão com o banco de dados

In [ ]:
db = sqlite3.connect(r'../../99 Datasets/datasets.db')
query = 'SELECT * FROM house_sales'

df = pd.read_sql_query(query, db)

# Primeira olhada nas informações e estatísticas descritivas dos dados

In [ ]:
df.info()

In [ ]:
df.describe()

# Explorando a correlação entre as variáveis

In [ ]:
# Plotando um mapa de calor das correlações com todas as variáveis

corrmat = df.corr()
cols = corrmat.nlargest(10, 'price')['price'].index
cm = np.corrcoef(corrmat.values.T)
sns.set(font_scale=1.15)
f, ax = plt.subplots(figsize=(15, 10))
hm = sns.heatmap(corrmat, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=corrmat.columns, xticklabels=corrmat.columns)

In [ ]:
# Identificando as 10 variáveis que mais estão mais correlacionadas com o PRICE

corrmat = df.corr()
cols = corrmat.nlargest(11, 'price')['price'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.15)
f, ax = plt.subplots(figsize=(10, 8))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)

In [ ]:
# Plotando o PAIPLOT para as variáveis mais correlacionadas com um sample (100 amostras)
sample = df[cols].sample(100)
sns.pairplot(sample)

## Imagem do mapa da região com as casas mais caras em cor mais escura

In [ ]:
from IPython.display import Image
Image(filename=r'img\houses_tableau.jpg')

# Primeiro modelo de regressão para usarmos de benchmark para os próximos 

In [ ]:
list(df.columns)

In [ ]:
function1 = '''
price ~
+ num_bed
+ num_bath
+ size_house
+ size_lot
+ num_floors
+ is_waterfront
+ C(condition)
+ size_basement
+ year_built
+ renovation_date
+ zip
+ latitude
+ longitude
+ avg_size_neighbor_houses
+ avg_size_neighbor_lot
'''

model1 = smf.ols(function1, df).fit()
print(model1.summary2())

## Explorando a variável CONDITION

In [ ]:
df.condition.value_counts(3)

In [ ]:
df.condition.hist()

# Criação de variáveis usando pandas

## Renovation Date

Ums boa forma de explorar a variável da data de renovação é criar uma DUMMY que indique se a casa foi renovada ou não - para isso, utilizamos o múdulo Numpy.Where, que traz uma condição para a existência de um renovation_date maior que zero como um valor 1 (True), e caso contrário um valor 0 (False)

In [ ]:
df['renovation_date'].value_counts(dropna=False).head()

In [ ]:
# Criando uma DUMMY para representar se a casa foi ou não reformada

df['dummy_reforma'] = np.where(df["renovation_date"]>0, 1, 0)

## Tempo desde a última reforma

In [ ]:
df['renovation_date'].max()

In [ ]:
# Criando uma outra DUMMY para representar o tempo em anos desde a última reforma

df['tempo_ultima_reforma'] = df['renovation_date'].max() - df['renovation_date']

In [ ]:
df.head()

## Método mais genérico com uma função e um apply

Criando uma função para retornar a subtração do valor máximo pelo ano da data de renovação, e 99 caso a data de renovação seja 0 - depois disso a função é aplicada para a criação de uma nova variável para traduzir o tempo da última reforma.

In [ ]:
%%time

max_date = df['renovation_date'].max()

def subtrai_coluna_com_se(row):
    
    if row['renovation_date'] == 0:
        return 99
    else:
        return (max_date - row['renovation_date'])


df['tempo_ultima_reforma'] = df.apply(subtrai_coluna_com_se, axis = 1)

In [ ]:
df['tempo_ultima_reforma'].value_counts().head()

## Criando mais variáveis a partir da data de renovação das casas

A intenção é explicar da melhor forma o preço das casas, com baase nos dados existentes - a partir daí o cientista de dados tem que ser criativo e assetrivo nas escolhar das novas variáveis para a criação do melhor modelo para explicação e predição do preço das casas.

In [ ]:
#A variável #renovated? (yes or no) é binária e mostra se ela foi ou nunca foi reformada 

df["renovated?"] = df["renovation_date"].apply(lambda x: 1 if x!=0 else 0)

In [ ]:
df["renovated?"].value_counts()

In [ ]:
#A variável recent_year coloca nela qual é o ano do imóvel, sendo que considera o ano de renovação caso haja

df["recent_year"]=0

df["recent_year"] = df.loc[df["renovation_date"]==0,"year_built"]

df.loc[df["renovation_date"] != 0,"recent_year"]=df.loc[df["renovation_date"] != 0 ,"renovation_date"]

In [ ]:
df["recent_year"].value_counts().head()

In [ ]:
#A variável existence_year calcula a diferença entre o ano mais recente da base(2015) e a idade do imóvel (recent_year) 

df["existence_year"]=2015-df["recent_year"]

In [ ]:
df["existence_year"].value_counts().head()

In [ ]:
#drop old values

df=df.drop(columns=["year_built","renovation_date"])

In [ ]:
df.head()

In [ ]:
df.info()

# Analisando novamente as correlações para continuar o trabalho de criação de variáveis e modelagem estatística

SELECT zip, avg(price) as avg_price
from dataset_house
group by zip

In [ ]:
df.corr()['price'].sort_values(ascending=False)

## Criando uma nova variável retirando o último dígito do ZIP das casas

Podemos diminuir a dimensão da variável ZIP pegando os primeiros quatro números

In [ ]:
df['newcep'] = df.zip.astype(str).str[:4]

## Criando uma variável de tamanho das casas do bairro

Para isso agruparemos os ZIPs calculando a média dos tamanhos das casas para cada um, criando uma nova coluna com o tamanho médio. Vamos usar funções no SQL e no pandas.

In [ ]:
# Identificando os valores dos preços médios por ZIP com uma função FOR

for cep in df.zip.unique():
    temp = df[df.zip==cep]
    print(cep,temp['price'].mean())

In [ ]:
df_zip = df.groupby(['zip']).agg({'size_house':'mean'})
df_zip.shape

In [ ]:
df_zip.info()

Para aplicar o tamanho médio das casas por ZIP devemos fazer um MERGE - código no SQL 

SELECT *
FROM df 
    inner join df_zip ON df_zip.index = df.zip
    
    
    
Seguindo com o código em Pandas

In [ ]:
df = pd.merge(df, df_zip, how='inner', left_on='zip', right_index=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

## Criando uma variável para fortalecer o preço das casas que são maiores que a média das casas do bairro

Após o MERGE dos DataFrames (original e o df_zip), a coluna size_house passou a ser chamada de size_house_x e o tamanho médio das casas do mesmo bairro passou a ser size_house_y.

Tendo a média dos valores da vizinhança podemos criar uma variavel de quanto a casa é maior do que a média das casas da vizinhança e criamos um indice com isso.

In [ ]:
df['indice_invejinha'] = df['size_house_x'] / df['size_house_y']

In [ ]:
df['indice_invejinha'].plot.hist(bins=30, figsize=(8,6))

In [ ]:
# function1 = '''
# price ~ 
#  + size_house
#  + num_bath
#  + size_house
#  + size_lot
#  + num_floors
#  + is_waterfront
#  + year_built
#  + latitude
#  + indice_invejinha
#  + longitude
#  + avg_size_neighbor_houses
#  + avg_size_neighbor_lot
#  + C(condition)
#  + C(zip)
# '''

# model1 = smf.ols(function1, df).fit()
# print(model1.summary2())

In [ ]:
df.corr()['price'].sort_values(ascending=False)

# Heterocedasticidade

<br>
<img src="img/heterocedasticidade.png" width="450" />
<br>

Heteroscedasticidade ou Heterocedasticidade é o fenômeno estatístico que ocorre quando o modelo de hipótese matemático apresenta variâncias para Y e X(X1, X2, X3,..., Xn) não iguais para todas as observações, contrariando o postulado : $E(u^2)=σ^2; i = 1 , 2 , ⋯ + n $

Esta hipótese do Modelo Clássico de Regressão Linear, pressupõe que a variância de cada termo de perturbação $u_i$, condicional aos valores escolhidos das variáveis explicativas, é algum número constante igual a $σ^2$.Ou seja, este postulado é a da homoscedasticidade, ou igual (homo) dispersão (scedasticidade), isto é, igual variância.

Em outras palavras, a heterocedasticidade apresenta-se como uma forte dispersão dos dados em torno de uma reta; uma dispersão dos dados perante um modelo econométrico regredido.

Uma definição mais precisa seria na qual uma distribuição de frequência em que todas as distribuições condicionadas têm desvios padrão diferentes.

O contrário desse fenômeno, a homocedasticidade, se dá pela observância do postulado, isto é, os dados regredidos encontram-se mais homogeneamente e menos dispersos (concentrados) em torno da reta de regressão do modelo.

Sua detecção pode ser realizada por meio do Teste de White, que consiste num teste residual. 

A heteroscedasticidade não elimina as propriedades de inexistência de viés e consistência dos estimadores de MQO, no entanto, eles deixam de ter variância mínima e eficiência, ou seja, não são os melhores estimadores lineares não-viesados (MELNV).

As medidas corretivas não são fáceis de serem implementadas. Se a amostra for grande, podemos obter os erros padrão com heteroscedasticidade corrigida segundo White dos estimadores de MQO e realizar inferências estatísticas com base nesses erros padrão. Por exemplo, no software Eviews, esta opção está disponível no menu quick, estimate equation, options e então seleciona-se a opção Heterokedasticity consistent coeficient covariance, White.

Diferentemente, se olharmos os resíduos de MQO, podemos levantar hipóteses sobre o provável padrão da heteroscedasticidade e transformar os dados originais de tal forma que não haja heteroscedasticidade nos dados transformados.

É comum seu acontecimento quando de pesquisas com dados em corte, ou seção transversal (cross section - observações de dados sobre unidades econômicas de diferentes tamanhos). 



# Homocedasticidade

Na estatística, uma sequência ou um vetor de variáveis aleatórias é homoscedástico se todas as variáveis aleatórias na sequência ou vetor tiverem a mesma variância finita. Isso também é conhecido como homogeneidade de variância. A noção complementar é chamada heterocedasticidade. As grafias homoscedasticidade e heteroscedasticidade também são usadas com frequência.

A suposição de homocedasticidade simplifica o tratamento matemático e computacional. Graves violações na homocedasticidade (supondo-se que a distribuição dos dados é homocedástica quando na realidade é heteroscedástica) pode resultar em superestimar a qualidade do ajuste medido pelo coeficiente de Pearson. Exemplo: Salario e poupança

## Aplicando os conceitos de Heterocedasticidade ao nosso exemplo

Analisando o primeiro scatterplot (preço x tamanho), notamos algumas variaveis com variância não constante, para torna-la homocedastica podemos aplicar diversas técnicas, usaremos o mais simples que é a aplicação de **LOG** nas duas variaveis pois além deconseguimos a variância constante, temos uma interpretação de elasticidade (taxa de variação) para os parâmetros.

**Vale ressaltar que a função LOG do Numpy está se referindo à operação de Logarítmo Natural, ou Logarítmo na base *e*, que é o número de Euller**

## Número de Euller

Na matemática, o número de Euler, denominado em homenagem ao matemático suíço Leonhard Euler, é a base dos logaritmos naturais. As variantes do nome do número incluem: número de Napier, número de Neper, constante de Néper, número neperiano, constante matemática, número exponencial etc. A primeira referência à constante foi publicada em 1618 na tabela de um apêndice de um trabalho sobre logaritmos de John Napier. No entanto, este não contém a constante propriamente dita, mas apenas uma simples lista de logaritmos naturais calculados a partir desta. A primeira indicação da constante foi descoberta por Jakob Bernoulli, quando tentava encontrar um valor para a expressão do cálculo de juros compostos, cujo valor é aproximadamente 2,718281828459045235360287. 

Abaixo a relação entre Preços e o tamanho da casa que imaginamos que tenha uma forte e positiva correlação.

In [ ]:
df[['size_house_x', 'price']].corr()

In [ ]:
# Gráfico de dispersão do preço das casas pelo seu tamanho

fig, ax = plt.subplots()
ax.scatter(x = df['size_house_x'], y = df['price'])
plt.ylabel('Price', fontsize=13)
plt.xlabel('Size', fontsize=13)
plt.show()

O formato do gráfico em funil é um clássico exemplo de Heterocedasticidade

## LOG: linearização corrige tanto a não linearidade, ameniza outliners e heterocedasticidade. 

Não necessáriamente precisamos ter distribuições normais para nossas amostras, mas ela ter essa caracteristica permite que façamos analises não só mais eficientes mas principalmente mais robustas já que a maioria dos algoritmos de regressões que usaremos trazem betas significativos para qualquer distribuição apenas com o primeiro momento, mas não somos capazes de fazer testes de hipótese sem o segundo momento.



In [ ]:
# Plotando o gráfico de distribuição dos preços das casas com a aproximação da curva normal

sns.distplot(df['price'] , fit=stats.norm);

(mu, sigma) = stats.norm.fit(df['price'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')


## Gráfico Q-Q

<br>
<img src="img/qqplot.png" width="450" />
<br>

*Origem: Wikipédia, a enciclopédia livre.*

Um gráfico Q-Q dados exponenciais independentes e randomicamente gerados, (X ~ Exp(1)). Este gráfico Q–Q compara uma amostra de dados no eixo vertical a uma estatística de população no eixo horizontal. Os pontos seguem um forte padrão não linear, sugerindo que os dados não são distribuídos com um padrão normal (X ~ N(0,1)). O deslocamento entre a linha e os pontos sugere que a média dos dados não é 0. A mediana dos pontos pode ser determinada a estar perto de 0,7
Gráfico Q-Q normal comparando dados normais independentes gerados aleatoriamente no eixo vertical a uma população normal padrão no eixo horizontal. A linearidade dos pontos sugere que os dados são normalmente distribuídos.

Em estatística, um gráfico Q-Q[1] ("Q" significa quantil) é um gráfico de probabilidades, que é um método gráfico para comparar duas distribuições de probabilidade, traçando seus quantis uns contra os outros. Primeiro, o conjunto de intervalos para os quantis é escolhido. Um ponto (x, y) no gráfico corresponde a um dos quantis da segunda distribuição (coordenada y) plotadas contra o mesmo mesmo quantil da primeira distribuição de (coordenada x). Portanto, a linha é uma curva paramétrica com o parâmetro que é o (número do) intervalo para quantil.

Se as duas distribuições que estão sendo comparadas são semelhantes, os pontos no gráfico Q-Q vai repousar na linha y = x, aproximadamente. Se as distribuições são linearmente relacionadas, os pontos no gráfico Q-Q irão repousar em uma linha, aproximadamente, mas não necessariamente na linha y = x. gráficos Q-Q também podem ser usados como meio gráfico de estimativa de parâmetros de dispersão e tendência central em uma família de distribuições.

Um gráfico Q-Q é usado para comparar as formas de distribuições, fornecendo uma exibição gráfica de como as propriedades, tais como medidas de tendência central, dispersão e assimetria são semelhantes ou diferentes nas duas distribuições. gráficos Q-Q podem ser usados para comparar conjuntos de dados ou distribuições teóricas. O uso de gráficos Q-Q para comparação de duas amostras de dados pode ser visto como uma abordagem não-paramétrica para comparação de suas distribuições subjacentes. Um gráfico Q-Q geralmente é uma abordagem mais poderosa para fazer isso do que a técnica comum de comparação de histogramas das duas amostras, mas requer mais habilidade para interpretar. Gráficos Q-Q são comumente usados para comparar um conjunto de dados com um modelo teórico.[2] Isto pode fornecer uma avaliação de "qualidade de ajuste" que é gráfica, ao invés de reduzir a uma exibição numérica. gráficos Q-Q também são usados para comparar duas distribuições teóricas entre si. Uma vez que gráficos Q-Q compararam distribuições, não há necessidade para os valores a serem observados como pares, como em um gráfico de dispersão, ou mesmo para o número de valores nos dois grupos sendo comparados ser igual.

O termo "gráfico de probabilidades" às vezes, refere-se especificamente a um gráfico Q-Q, umas vezes a uma classe gráficos e outras para o menos comumente usado gráfico P-P. O coeficiente de correlação do gráfico de probabilidade é uma grandeza derivada da ideia de gráficos Q-Q, que mede a concordância de uma distribuição ajustada com os dados observados e que às vezes é usada como um meio de ajuste de uma distribuição de dados. 

In [ ]:
# Plotando o gráfico de probabilidade dos preços das casas
fig = plt.figure()
res = stats.probplot(df['price'], plot=plt)
plt.show()

## Aplicando log natural aos preços e aos tamanhos das casas

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = np.log(df['size_house_x']), y = np.log(df['price']))
plt.ylabel('Price', fontsize=13)
plt.xlabel('Size', fontsize=13)
plt.show()


sns.distplot(np.log(df['price']) , fit=stats.norm);

(mu, sigma) = stats.norm.fit(np.log(df['price']))
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')

fig = plt.figure()
res = stats.probplot(np.log(df['price']), plot=plt)
plt.show()

# A segunda variável com maior correlação é a média de tamanho das casas da vizinhança.

Uma forma alternativa de transformação dessa informação em uma variavel explicativa interessante é considerar não exatamente o tamanho médio das casas do bairro, mas o quanto estamos proximos ou longe da média das casas do bairro, pois a média pode ser grande ou pequena, mas nossa casa pode ser ainda maior ou ainda menor, não fazendo a comparação relativa. 

Portanto vamos analisar a variável, aplicar a LOG-Linearização e criar uma nova variável que será uma proporção entre a venda e a sua média.

Obs: Não faria sentido criar novas variaveis que fossem combinações lineares entre as variaveis do modelo, muitos softwares simplesmente não rodam por não conseguirem inveter a matriz de parametros (X) acusando multicolinearidade perfeita (ou em termos mais economicos, a variavel criada como combinação linear de outras não acrescenta nenhum novo poder explicativo ao modelo), neste casso criaremos um indice percentual para testarmos sua correlação com o preço.

In [ ]:
# Plotando o gráfico de distribuição dos tamanhos médios das casas da vizinhança com a aproximação da curva normal

sns.distplot(df['avg_size_neighbor_houses'] , fit=stats.norm);

(mu, sigma) = stats.norm.fit(df['avg_size_neighbor_houses'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('avg_size_neighbor_houses')

# Gráfico de dispersão dos tamanhos médios das cadas da vizinhança pelo preço
fig, ax = plt.subplots()
ax.scatter(x = df['avg_size_neighbor_houses'], y = df['price'])
plt.ylabel('Price', fontsize=13)
plt.xlabel('neighbor', fontsize=13)
plt.show()

# Gráfico de probabilidades dos tamanhos médios das cadas da vizinhança
res1 = stats.probplot(df['avg_size_neighbor_houses'], plot=plt)
plt.show()



## Aplicando log natural aos tamanhos médios das cadas da vizinhança

In [ ]:
# Gráfico de distribuição com LOG aos tamanhos médios das cadas da vizinhança
sns.distplot(np.log(df['avg_size_neighbor_houses']) , fit=stats.norm);

(mu, sigma) = stats.norm.fit(np.log(df['avg_size_neighbor_houses']))
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('avg_size_neighbor_houses')

# Gráfico de dispersão dos tamanhos médios das cadas da vizinhança pelo preço
fig, ax = plt.subplots()
ax.scatter(x = np.log(df['avg_size_neighbor_houses']), y = np.log(df['price']))
plt.ylabel('Price', fontsize=13)
plt.xlabel('neighbor', fontsize=13)
plt.show()

#Gráfico de probabilidades dos tamanhos médios das cadas da vizinhança
res2 = stats.probplot(np.log(df['avg_size_neighbor_houses']), plot=plt)
plt.show()

## Como escolher as variáveis que deveriamos aplicar LOG

Uma distribuição normal tem 4 momentos como as outras, mas só precisamos definir os 2 primeiros:

- Média 
- Desvio-Padrão

os outros dois momentos, assimetria e curtose, são fixos para qualquer curva normal

- Assimetria = 0 
- Curtose = 3 

Uma forma mais analitica é vermos quais são esses dois parâmetros da nossa distribuição, e caso estejam muito distante do padrão, a variável é uma boa candidata a log-linearização.

In [ ]:
# Calculando a assimetria (skew) para as variáveis independentes e colocando em um DataFrame

numeric_feats = df.dtypes[df.dtypes != "object"].index

skewed_feats = df[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

Segundo estas informações, todas as variáveis con assimetria acima de 3 poderiam ser linearizadas pela aplicação do LOG natural.

# Variáveis Candidatas a Categóricas

## Dummy variable (statistics)

<br>
<img src="img/dummies.png" width="450" />
<br>

*Origem: Wikipédia, a enciclopédia livre*

Em estatística e econometria, particularmente na análise de regressão, uma variável dummy (também conhecida como variável indicadora, variável de design, codificação de um hot, indicador booleano, variável binária, ou variável qualitativa) é aquela que leva a valor 0 ou 1 para indicar a ausência ou presença de algum efeito categórico que pode ser esperado para mudar o resultado. Variáveis ​​dummy são usadas como dispositivos para classificar os dados em categorias mutuamente exclusivas (como fumante / não fumante, etc.). Por exemplo, na análise econométrica de séries temporais, variáveis ​​dummy podem ser usadas para indicar a ocorrência de guerras ou grandes greves. Uma variável dummy pode, assim, ser considerada como um valor de verdade representado como um valor numérico 0 ou 1 (como às vezes é feito na programação de computadores).

Variáveis ​​dummy são variáveis ​​"proxy" ou substitutos numéricos para fatos qualitativos em um modelo de regressão. Na análise de regressão, as variáveis ​​dependentes podem ser influenciadas não apenas por variáveis ​​quantitativas (renda, produto, preços, etc.), mas também por variáveis ​​qualitativas (gênero, religião, região geográfica, etc.). Uma variável independente fictícia (também chamada de variável explicativa fictícia) que para algumas observações tem um valor de 0 fará com que o coeficiente dessa variável não tenha nenhum papel em influenciar a variável dependente, enquanto quando o dummy assume um valor 1 seu coeficiente age para alterar a interceptação. Por exemplo, suponha que a associação em um grupo seja uma das variáveis ​​qualitativas relevantes para uma regressão. Se a associação ao grupo receber arbitrariamente o valor de 1, todos os outros receberão o valor 0. Então, o intercepto (o valor da variável dependente se todas as outras variáveis ​​explicativas assumiram hipoteticamente o valor zero) seria o termo constante para membros, mas seria o termo constante mais o coeficiente do manequim de associação no caso dos membros do grupo.

Variáveis ​​dummy são usadas freqüentemente na análise de séries temporais com mudança de regime, análise sazonal e aplicações de dados qualitativos. Variáveis ​​dummy estão envolvidas em estudos para previsão econômica, estudos biomédicos, pontuação de crédito, modelagem de resposta, etc. Variáveis ​​dummy podem ser incorporadas em métodos tradicionais de regressão ou paradigmas de modelagem desenvolvidos recentemente.

Análise das variáveis categóricas com sugestão de tratamento através da criação de ***DUMMIES*** de forma que sejam melhor interpretadas pelo modelo e representam da melhor forma a composição dos preços das casas

In [ ]:
df.head()

In [ ]:
list(df.columns)

## Analisando a variável do número de banheiros das casas

In [ ]:
df.num_bath.plot.hist(bins=25)

In [ ]:
df.num_bath.value_counts()

In [ ]:
# Criando uma coluna com o LOG dos preços das casas

df['log_price'] = np.log1p(df['price'])

In [ ]:
# Comparando graficamente a distribuição do número de banheiros das casas pelo preço

df.plot.scatter(x='num_bath', y='price')

df.plot.scatter(x='num_bath', y='log_price')

### Rodando regressões do número de banheiros das casas pelo preço

In [ ]:
# Regressão simples do número de banheiros pelo preço das casas

function2 = '''
price ~ num_bath
'''

model2 = smf.ols(function2, df).fit()
print(model2.summary2())

In [ ]:
# Regressão simples do número de banheiros pelo LOG do preço das casas

function2 = '''
np.log1p(price) ~ num_bath
'''

model2 = smf.ols(function2, df).fit()
print(model2.summary2())

In [ ]:
# Rodando uma regressão com o número de banheiros como DUMMIES pelo preço das casas

function2 = '''
price ~ C(num_bath)
'''

model2 = smf.ols(function2, df).fit()
print(model2.summary2())

In [ ]:
# Calculando as correlações entre os preços, LOG de preços e número de banheiros

df[['price', 'log_price', 'num_bath']].corr().round(2)

## Criando variáveis com as CATEGORIAS para número de banheiros

Serão atribuidos valores inteiros para a criação de novas variáveis que representam novas CATEGORIAS para número de banheiros e número parcial de banheiros, com limitação para os maiores valores pequeno número de incidências.

O objetivo é obter distribuições que sejam parecidas com a Normal, além de eliminar outliers e dessa forma possam melhor representar a composição dos preços das casas.

In [ ]:
# Plotando o gráfico de distribuição do número de banheiros
g = sns.factorplot(x="num_bath", data=df, kind="count",
                   palette="rainbow", size=6, aspect=1.5)
g.set_xticklabels(step=2)


# Criando uma nova DUMMY para o número de banheiros e plotando a distribuição
df['c_num_bath'] = df.num_bath.replace({0:0, 0.50:0, 0.75:0,
                                          1:1, 1.25:1, 1.50:1, 1.75:1,
                                          2:2, 2.25:2, 2.50:2, 2.75:2,
                                          3:3, 3.25:3, 3.50:3, 3.75:3,
                                          4:4, 4.25:4, 4.50:4, 4.75:4,
                                          5:5, 5.25:5, 5.50:5, 5.75:5,
                                          6:5, 6.25:5, 6.50:5, 6.75:5,
                                          7:5, 7.25:5, 7.50:5, 7.75:5,
                                          8:5})

g = sns.factorplot(x="c_num_bath", data=df, kind="count",
                   palette="rainbow", size=6, aspect=1.5)
g.set_xticklabels(step=2)


# Criando uma nova DUMMY para o número de banheiros PARCIAIS e plotando a deitribuição
df['c_num_partial_bath'] = df.num_bath.replace({0:0, 0.25:1, 0.50:2, 0.75:3,
                                                  1:0, 1.25:1, 1.50:2, 1.75:3,
                                                  2:0, 2.25:1, 2.50:2, 2.75:3,
                                                  3:0, 3.25:1, 3.50:2, 3.75:3,
                                                  4:0, 4.25:1, 4.50:2, 4.75:3,
                                                  5:0, 5.25:1, 5.50:2, 5.75:3,
                                                  6:0, 6.25:1, 6.50:2, 6.75:3,
                                                  7:0, 7.25:1, 7.50:2, 7.75:3,
                                                  8:0})

g = sns.factorplot(x="c_num_partial_bath", data=df, kind="count",
                   palette="rainbow", size=6, aspect=1.5)

g.set_xticklabels(step=2)

### Rodando a regressão com as novas variáveis para o número de banheiros

In [ ]:
function2 = '''
np.log1p(price) ~ C(c_num_bath) + C(c_num_partial_bath) 
'''

model2 = smf.ols(function2, df).fit()
print(model2.summary2())

## Analisando as variáveis de número de quartos

Da mesma forma que no número de banheiros, vamos criar uma **DUMMY** para o número de quantos que possa explicar melhor o preço das casas segundo o númeor de quartos.

In [ ]:
df.num_bed.value_counts()

In [ ]:
# Plotando um gráfico de distribuição para o número de quartos
g = sns.factorplot(x="num_bed", data=df, kind="count",
                   palette="rainbow", size=6, aspect=1.5)
g.set_xticklabels(step=2)


# Criando uma DUMMY para o número de quartos que explique melhor o preço das casas
df['c_num_bed'] = df.num_bed.replace({ 0:1, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:6, 8:6, 9:6, 10:6, 33:6})

g = sns.factorplot(x="c_num_bed", data=df, kind="count",
                   palette="rainbow", size=6, aspect=1.5)
g.set_xticklabels(step=2)

# Mudando a forma funcional das variáveis para capturar não-lineariedades

Vamos fazer uma função para a criação de colunas QUADRÁTICAS para cada uma das colunas numéricas do DataFrame - dessa forma podemos capturar não linearidades para as variações das colunas existentes.

In [ ]:
for coluna in list(df):
    try:
        df['q_'+str(coluna)] = df[coluna]**2
    except:
        pass

In [ ]:
df.head()

In [ ]:
df.corr()['log_price'].sort_values(ascending=False)

In [ ]:
# Redefinindo o tamanho da casa

df['q_size_house'] = df['size_house_x'] * df['size_house_x']

## Rodando a regressão com as novas variáveis

In [ ]:
function3 = ''' log_price ~ num_bed
+ num_bath
+ size_house_x
+ size_lot
+ num_floors
+ is_waterfront
+ condition
+ size_basement
+ zip
+ latitude
+ longitude
+ avg_size_neighbor_houses
+ avg_size_neighbor_lot
+ dummy_reforma
+ tempo_ultima_reforma
+ recent_year
+ existence_year
+ newcep
+ size_house_y
+ indice_invejinha
+ c_num_bath
+ c_num_partial_bath
+ c_num_bed
+ q_size_house

+ q_num_bed
+ q_num_bath
+ q_size_house_x
+ q_size_lot
+ q_num_floors
+ q_is_waterfront
+ q_condition
+ q_size_basement
+ q_zip
+ C(zip)
+ q_latitude
+ q_longitude
+ q_avg_size_neighbor_houses
+ q_avg_size_neighbor_lot
+ q_dummy_reforma
+ q_tempo_ultima_reforma
+ q_recent_year
+ q_existence_year
+ q_size_house_y
+ q_indice_invejinha
+ q_c_num_bath
+ q_c_num_partial_bath
+ q_c_num_bed

'''

In [ ]:
model3 = smf.ols(function3, df).fit()
print(model3.summary2())

## Explorando a influência do tamanho das casas na composição do preço

In [ ]:
function4 = '''
np.log1p(price) ~ size_house_x + q_size_house
'''

model4 = smf.ols(function4, df).fit()
print(model4.summary2())

## Verificando as correlações com o LOG dos preços

In [ ]:
corr = df.corr()
corr.sort_values(["log_price"], ascending = False, inplace = True)
print(corr.log_price)

# Realizando Predições

Agora vamso utilizaro o modelo OLS (Ordinary Least Squares) utilizado anteriormente para análise, para fazer predições dos preços das casas, e deppos analisar estes preços preditos e principalmente como o erro gerado (Valor Real menos o Valor Predito) se comporta.

In [ ]:
function5 = ''' price ~ num_bed
+ num_bath
+ size_house_x
+ size_lot
+ num_floors
'''

model5 = smf.ols(function5, df).fit()
print(model5.summary2())

In [ ]:
model5.predict()

In [ ]:
df['yhat'] = model5.predict()

In [ ]:
df[['yhat', 'price']].corr()

In [ ]:
np.sqrt(model4.rsquared)

In [ ]:
df['erro'] = df['yhat'] - df['price']

In [ ]:
from sklearn import metrics
mean_squared_error(df.price, df.yhat), r2_score(df.price, df.yhat)

# Analisando o erro

"O dinheiro está no residuo" M. Silva - isto significa que as melhores oportunidades estarão nas casas com menor preço real de venda mas com mais alto valor predito.

Além disso, temos que lembrar que o erro deve ter distribuição normal.

In [ ]:
df['erro'].plot.hist()

In [ ]:
df['erro'].plot.hist(bins=40, xlim = (-1000000, 1000000))

In [ ]:
df.price.mean()

In [ ]:
df['erro'].plot.kde(xlim = (-1000000, 1000000) )

In [ ]:
df.head()

## Normalizando o Erro

Dividindo os valores de erro pelo desvio padrão do próprio erro faz o que chamamos de Normalização dos valores do erro, para entendermos quantos desvios padrão estamos afastados do valor médio. Para os erros, o valor médio sempre será zero.

In [ ]:
df['erro_em_desvios'] = df['erro']/df['erro'].std()

In [ ]:
df['erro_em_desvios'].plot.kde(xlim = (-8, 6) )

# Removendo outliners da predição usando probabilidade

Podemos identificar que temos muitos valores extremos na distribuição dos erros, o que causa muita imprecisão. A estratégia de tratamento é a remoção destes Outliers. Vamos começar criando uma máscara para removeer os outliers com dois desvios-padrão de distância, e identificar quantos valores serão removidos.

In [ ]:
mascara_prob = (df['erro_em_desvios']<1.96) & (df['erro_em_desvios']>-1.96)

In [ ]:
df.shape

In [ ]:
df[~mascara_prob].shape

In [ ]:
df[mascara_prob].shape

In [ ]:
df[mascara_prob].shape[0]/df.shape[0]

A remoção de valores com dois desvios-padrão da média parece muito agressivo, pois estaremos eliminando cerca de 4% da amostra. Isso pode enviesar nossos dados. Por isso, vamos identificar quantos valores removemos se adoratmos uma estratégia de 6 sigma de nível de confiança.

In [ ]:
mascara_raridade_9999 = (df['erro_em_desvios']<6) & (df['erro_em_desvios']>-6)

In [ ]:
df[~mascara_raridade_9999].shape[0]

Esta estratégia 6 sigma nos permite retirar somente 48 valores extremos do modelo - agora sim seguimos em frente com a nova regressão.

In [ ]:
function6 = ''' price ~ num_bed
+ num_bath
+ size_house_x
+ size_lot
+ num_floors
'''

model6 = smf.ols(function6, df[mascara_raridade_9999]).fit()
print(model6.summary2())

# Analisando a DUMMY número de banheiros através de uma regressão

Podemos observar cada linha vermelha representando os valores preditos para cada número de banheiros

In [ ]:
function7 = ''' price ~ size_house_x + C(c_num_bath)
'''

model7 = smf.ols(function7, df).fit()
print(model7.summary2())

import matplotlib.pyplot as plt
import statsmodels.api as sm

plt.rcParams['figure.figsize'] = (9,9)

fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(model7, 6, ax=ax) # o parâmetro 6 é o índice da variável do modelo (size_house_x)
ax.set_ylabel("price")
ax.set_xlabel("size")
ax.set_title("Linear Regression")

# Acicionando ITERAÇÃO entre as variáveis parar capturar diferenças nas taxas de retorno

Vamos iteragir as varáveis is_waterfront com o soze_house_x com multiplicação para geraar novas predições

In [ ]:
df.is_waterfront.describe()

In [ ]:
df['size_x_waterfront'] = df['size_house_x'] * df['is_waterfront']

In [ ]:
function8 = ''' log_price ~ size_house_x + is_waterfront + size_x_waterfront'''

model8 = smf.ols(function8, df).fit()
print(model8.summary2())

In [ ]:
fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(model8, 1, ax=ax)
ax.set_ylabel("price")
ax.set_xlabel("size")
ax.set_title("Linear Regression")

## Rodando o mesmo gréfico para o modelo 5, com parâmetro 5 (size_house_x)

In [ ]:
model6.params

In [ ]:
import statsmodels.api as sm

fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(model6, 3, ax=ax)
ax.set_ylabel("price")
ax.set_xlabel("size")
ax.set_title("Linear Regression")

# Rodando o mesmo gráfico para o modelo 1

In [ ]:

sm.graphics.plot_fit(model2, 2, ax=ax)

In [ ]:
model2.params

# Teste de normalidade do erro para o Modelo 1


In [ ]:
model1.resid.plot.hist(figsize=(12,8), bins=100, xlim=(-1000000,1500000))

In [ ]:
model1.predict()

In [ ]:
df['price'].values

In [ ]:
results = list(zip(model1.predict(),df['price'].values))

In [ ]:
pd.DataFrame(results, columns=['yhat', 'y']).plot.scatter(x='yhat', y='y', figsize=(12,8), xlim=(0,2000000), ylim=(0,2000000))

In [ ]:
results = pd.DataFrame(results, columns=['yhat', 'y'])
results['residuo'] = model1.resid


In [ ]:
results.head()

Na análise de resíduo, diferença entre o valor predito e o valor y teste, nota-se que ela está distribuida aleatoriamente, indicando distribuição normal dos resultados e assim, pode-se dizer que o modelo possui poder explicativo satisfatório. No entanto, o teste de Shapiro rejeita a hipótese de normalidade dos resíduos. Dessa forma, ainda que o modelo apresente pontuação razoavelmente alto de 84~85, ainda há pontos para serem melhorados afim de melhorar a distribuicao residual.


In [ ]:

plt.rcParams['figure.figsize'] = (9,9)

results["residuo2"] = results["y"] - results["yhat"]
results.plot(x = "yhat", y = "residuo",kind = "scatter")

t,p=stats.shapiro(results["residuo"])

print("The Shapiro-test of normality for residuals is {:03.3f} and p-value of {:03.3f}".format(t,p))

In [ ]:
stats.shapiro(results["residuo2"])